In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
import numpy as np
import random
import json
import zipfile
import io
import math
import time
from copy import deepcopy

# ============================================================================
# 1. HELPERS & ENCODERS
# ============================================================================
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        if isinstance(obj, np.floating): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

# ============================================================================
# 2. NIM ENVIRONMENT
# ============================================================================
class Nim:
    def __init__(self, piles=[1, 3, 5, 7]):
        self.initial_piles = piles[:]
        self.reset()
    
    def reset(self):
        self.piles = self.initial_piles[:]
        self.current_player = 0
        self.game_over = False
        self.winner = None
        self.move_history = []
        return self.get_state()
    
    def get_state(self):
        # Return hashable state representation (sorted tuple)
        return tuple(sorted(self.piles))
    
    def get_available_actions(self):
        actions = []
        for i, pile in enumerate(self.piles):
            for take in range(1, pile + 1):
                actions.append((i, take))
        return actions
    
    def make_move(self, action):
        if self.game_over: return self.get_state(), 0, True
        
        pile_idx, take = action
        if pile_idx >= len(self.piles) or take > self.piles[pile_idx] or take < 1:
            return self.get_state(), -100, True
        
        self.piles[pile_idx] -= take
        self.move_history.append((self.current_player, action))
        
        # Check lose condition (Last stick taker loses)
        if sum(self.piles) == 0:
            self.game_over = True
            self.winner = 1 - self.current_player
            return self.get_state(), -50, True
        
        self.current_player = 1 - self.current_player
        return self.get_state(), 0, False
    
    def copy(self):
        new_env = Nim(self.initial_piles)
        new_env.piles = self.piles[:]
        new_env.current_player = self.current_player
        new_env.game_over = self.game_over
        new_env.winner = self.winner
        return new_env

# ============================================================================
# 3. MCTS NODE
# ============================================================================
class MCTSNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0.0
        self.untried_actions = None
    
    def uct_value(self, exploration=1.41):
        if self.visits == 0: return float('inf')
        return self.value / self.visits + exploration * math.sqrt(math.log(self.parent.visits) / self.visits)
    
    def best_child(self, exploration=1.41):
        return max(self.children, key=lambda c: c.uct_value(exploration))

# ============================================================================
# 4. NIM AGENT
# ============================================================================
class NimAgent:
    def __init__(self, player_id, lr=0.1, gamma=0.95, epsilon=1.0,
                 epsilon_decay=0.995, epsilon_min=0.01, mcts_sims=50, minimax_depth=3):
        self.player_id = player_id; self.lr = lr; self.gamma = gamma
        self.epsilon = epsilon; self.epsilon_decay = epsilon_decay; self.epsilon_min = epsilon_min
        self.mcts_simulations = mcts_sims; self.minimax_depth = minimax_depth
        self.q_table = {}
        self.wins = 0; self.losses = 0; self.draws = 0; self.total_score = 0; self.games_played = 0
    
    def get_q_value(self, state, action):
        action_key = str(action)
        return self.q_table.get((state, action_key), 0.0)
    
    def choose_action(self, env, training=True):
        available_actions = env.get_available_actions()
        if not available_actions: return None
        if training and random.random() < self.epsilon: return random.choice(available_actions)
        
        # Combined Strategy
        best_action = None; best_score = -float('inf')
        for action in available_actions:
            score = 0.0
            score += self.get_q_value(env.get_state(), action) * 0.3 # Q-Value
            if self.mcts_simulations > 0: score += self._evaluate_action_mcts(env, action) * 0.4 # MCTS
            if self.minimax_depth > 0: score += self._evaluate_action_minimax(env, action) * 0.3 # Minimax
            if score > best_score: best_score = score; best_action = action
        return best_action if best_action else available_actions[0]
    
    def _evaluate_action_mcts(self, env, action):
        sim = env.copy(); sim.make_move(action)
        if sim.game_over: return 100 if sim.winner == self.player_id else -100
        root = MCTSNode(sim.get_state()); root.untried_actions = sim.get_available_actions()[:]
        for _ in range(self.mcts_simulations):
            node = root; temp = sim.copy()
            while node.untried_actions == [] and node.children:
                node = node.best_child(); 
                if node.action: temp.make_move(node.action)
            if node.untried_actions and not temp.game_over:
                na = random.choice(node.untried_actions); node.untried_actions.remove(na)
                temp.make_move(na); child = MCTSNode(temp.get_state(), parent=node, action=na)
                node.children.append(child); node = child
            
            # Simulation
            rw = 0; steps = 0
            t_sim = temp.copy()
            while not t_sim.game_over and steps < 30:
                 acts = t_sim.get_available_actions()
                 if not acts: break
                 t_sim.make_move(random.choice(acts)); steps += 1
            if t_sim.game_over: rw = 50 if t_sim.winner == self.player_id else -50
            
            # Backprop
            while node: node.visits += 1; node.value += rw; node = node.parent
        return root.value / max(root.visits, 1)
    
    def _evaluate_action_minimax(self, env, action):
        sim = env.copy(); sim.make_move(action)
        if sim.game_over: return 100 if sim.winner == self.player_id else -100
        return self._minimax(sim, self.minimax_depth - 1, -float('inf'), float('inf'), False)
    
    def _minimax(self, env, depth, alpha, beta, is_maximizing):
        if env.game_over: return 100 if env.winner == self.player_id else -100
        if depth == 0: return 0
        actions = env.get_available_actions()
        if not actions: return 0
        if is_maximizing:
            me = -float('inf')
            for a in actions[:10]:
                s = env.copy(); s.make_move(a)
                ev = self._minimax(s, depth - 1, alpha, beta, False)
                me = max(me, ev); alpha = max(alpha, ev)
                if beta <= alpha: break
            return me
        else:
            me = float('inf')
            for a in actions[:10]:
                s = env.copy(); s.make_move(a)
                ev = self._minimax(s, depth - 1, alpha, beta, True)
                me = min(me, ev); beta = min(beta, ev)
                if beta <= alpha: break
            return me

    def update_q_value(self, state, action, reward, next_state, next_actions):
        ak = str(action)
        cq = self.get_q_value(state, action)
        mnq = max([self.get_q_value(next_state, a) for a in next_actions], default=0) if next_actions else 0
        self.q_table[(state, ak)] = cq + self.lr * (reward + self.gamma * mnq - cq)
    
    def decay_epsilon(self): self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    def reset_stats(self): self.wins = 0; self.losses = 0; self.total_score = 0; self.games_played = 0

def play_game(env, agent1, agent2, training=True):
    env.reset()
    agents = [agent1, agent2]
    moves = 0
    while not env.game_over and moves < 50:
        cp = env.current_player; agent = agents[cp]
        s = env.get_state()
        a = agent.choose_action(env, training)
        if a is None: break
        ns, r, done = env.make_move(a)
        if training: agent.update_q_value(s, a, r, ns, env.get_available_actions())
        moves += 1
        if done and env.winner is not None:
            agents[env.winner].wins += 1; agents[1-env.winner].losses += 1
            for ag in agents: ag.games_played += 1
    return env.winner

In [22]:
import time
import json
import zipfile

# ============================================================================
# 1. SERIALIZATION 
# ============================================================================
def serialize_q_table(q_table):
    serialized_q = {}
    for (state, action_key), value in q_table.items():
        state_str = json.dumps(list(state))
        key_str = f"{state_str}|{action_key}"
        serialized_q[key_str] = float(value)
    return serialized_q

def save_kaggle_brain(agent1, agent2, config, filename="nim_brains.zip"):
    # Construct data dictionaries 
    agent1_data = {
        "q_table": serialize_q_table(agent1.q_table),
        "epsilon": float(agent1.epsilon),
        "wins": int(agent1.wins), "losses": int(agent1.losses),
        "total_score": int(agent1.total_score),
        "games_played": int(agent1.games_played),
        "mcts_simulations": int(agent1.mcts_simulations),
        "minimax_depth": int(agent1.minimax_depth)
    }
    agent2_data = {
        "q_table": serialize_q_table(agent2.q_table),
        "epsilon": float(agent2.epsilon),
        "wins": int(agent2.wins), "losses": int(agent2.losses),
        "total_score": int(agent2.total_score),
        "games_played": int(agent2.games_played),
        "mcts_simulations": int(agent2.mcts_simulations),
        "minimax_depth": int(agent2.minimax_depth)
    }
    
    print(f"💾 Saving: A1 Q-States={len(agent1.q_table)}, A2 Q-States={len(agent2.q_table)}")
    
    with zipfile.ZipFile(filename, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.writestr("agent1.json", json.dumps(agent1_data, cls=NumpyEncoder, indent=2))
        zf.writestr("agent2.json", json.dumps(agent2_data, cls=NumpyEncoder, indent=2))
        zf.writestr("config.json", json.dumps(config, cls=NumpyEncoder, indent=2))
    
    print(f"✅ Save Complete! Download '{filename}'")

# ============================================================================
# 2. TRAINING RUNNER (Now with HISTORY!)
# ============================================================================
def run_nim_training():
    # --- CONFIGURATION ---
    EPISODES = 100     
    PILES = [1, 3, 5, 7] 
    
    env = Nim(PILES)
    
    # === ADJUST BRAIN POWER HERE ===
    agent1 = NimAgent(0, lr=0.1, gamma=0.95, epsilon_decay=0.9995, 
                      mcts_sims=200, minimax_depth=10)
    agent2 = NimAgent(1, lr=0.1, gamma=0.95, epsilon_decay=0.9995, 
                      mcts_sims=200, minimax_depth=10)
    
    # --- FIX: Initialize History Lists ---
    history = {
        'agent1_wins': [], 'agent2_wins': [],
        'agent1_epsilon': [], 'agent2_epsilon': [],
        'agent1_q_size': [], 'agent2_q_size': [],
        'episode': []
    }
    
    print(f"🚀 Starting HIGH-IQ Nim Training ({EPISODES} Episodes)...")
    start_time = time.time()
    
    for ep in range(1, EPISODES + 1):
        play_game(env, agent1, agent2, training=True)
        
        agent1.decay_epsilon()
        agent2.decay_epsilon()
        
        # Record stats every 50 episodes (Matches Streamlit default)
        if ep % 5 == 0:
            history['agent1_wins'].append(agent1.wins)
            history['agent2_wins'].append(agent2.wins)
            history['agent1_epsilon'].append(agent1.epsilon)
            history['agent2_epsilon'].append(agent2.epsilon)
            history['agent1_q_size'].append(len(agent1.q_table))
            history['agent2_q_size'].append(len(agent2.q_table))
            history['episode'].append(ep)
            
        if ep % 5 == 0:
            elapsed = time.time() - start_time
            print(f"Episode {ep}/{EPISODES} | A1 Wins: {agent1.wins} | ε={agent1.epsilon:.4f} | {elapsed:.1f}s")
    
    # Save the FULL history dictionary
    config = {
        "lr1": agent1.lr, "gamma1": agent1.gamma,
        "lr2": agent2.lr, "gamma2": agent2.gamma,
        "piles": PILES,
        "training_history": history 
    }
    
    print("\n🏆 Training Finished!")
    save_kaggle_brain(agent1, agent2, config)

if __name__ == "__main__":
    run_nim_training()

🚀 Starting HIGH-IQ Nim Training (100 Episodes)...
Episode 5/100 | A1 Wins: 1 | ε=0.9975 | 0.0s
Episode 10/100 | A1 Wins: 3 | ε=0.9950 | 0.0s
Episode 15/100 | A1 Wins: 6 | ε=0.9925 | 0.0s
Episode 20/100 | A1 Wins: 9 | ε=0.9900 | 10.5s
Episode 25/100 | A1 Wins: 13 | ε=0.9876 | 10.5s
Episode 30/100 | A1 Wins: 16 | ε=0.9851 | 10.6s
Episode 35/100 | A1 Wins: 19 | ε=0.9826 | 10.6s
Episode 40/100 | A1 Wins: 23 | ε=0.9802 | 10.6s
Episode 45/100 | A1 Wins: 27 | ε=0.9777 | 10.6s
Episode 50/100 | A1 Wins: 30 | ε=0.9753 | 16.2s
Episode 55/100 | A1 Wins: 33 | ε=0.9729 | 16.2s
Episode 60/100 | A1 Wins: 35 | ε=0.9704 | 16.2s
Episode 65/100 | A1 Wins: 36 | ε=0.9680 | 21.8s
Episode 70/100 | A1 Wins: 39 | ε=0.9656 | 27.5s
Episode 75/100 | A1 Wins: 42 | ε=0.9632 | 27.5s
Episode 80/100 | A1 Wins: 46 | ε=0.9608 | 27.9s
Episode 85/100 | A1 Wins: 48 | ε=0.9584 | 33.5s
Episode 90/100 | A1 Wins: 52 | ε=0.9560 | 33.7s
Episode 95/100 | A1 Wins: 55 | ε=0.9536 | 33.7s
Episode 100/100 | A1 Wins: 59 | ε=0.9512 | 39.